In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

City_ID          City Country      Lat       Lng  Max Temp  Humidity  \
0        0       Mataura      NZ -46.1927  168.8643     48.47        78   
1        1  Saint George      US  37.1041 -113.5841     95.63        11   
2        2       Ushuaia      AR -54.8000  -68.3000     28.06       100   
3        3        Barrow      US  71.2906 -156.7887     35.62        93   
4        4      Khatanga      RU  71.9667  102.5000     36.43        99   

   Cloudiness  Wind Speed Current Description  
0          99        3.27     overcast clouds  
1           1        1.99           clear sky  
2          75       17.27          heavy snow  
3          90       11.50          light rain  
4         100       12.88          light rain

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

City_ID          City Country      Lat       Lng  Max Temp  Humidity  \
5         5         Airai      TL  -8.9266  125.4092     76.37        44   
9         9         Vaini      TO -21.2000 -175.2000     82.56        83   
11       11       Merauke      ID  -8.4667  140.3333     84.90        55   
15       15        Atuona      PF  -9.8000 -139.0333     77.88        75   
17       17       Rikitea      PF -23.1203 -134.9692     73.94        67   
19       19      Qui Nhon      VN  13.7667  109.2333     82.85        77   
23       23  Ponta Do Sol      PT  32.6667  -17.1000     74.71        70   
26       26      Acapulco      MX  16.8634  -99.8901     82.22        78   
30       30     Caravelas      BR -17.7125  -39.2481     74.46        86   
38       38     San Borja      BO -14.8167  -66.8500     75.96        68   

    Cloudiness  Wind Speed Current Description  
5           56        5.37       broken clouds  
9           40       11.50    scattered clouds  
11          72       11.21       broken clouds  
15           5       19.95           clear sky  
17           1       14.34           clear sky  
19         100        3.11     overcast clouds  
23          58        5.23       broken clouds  
26          75        6.91       broken clouds  
30           0        9.48           clear sky  
38          22        2.84          few clouds

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

City Country  Max Temp Current Description      Lat       Lng  \
5          Airai      TL     76.37       broken clouds  -8.9266  125.4092   
9          Vaini      TO     82.56    scattered clouds -21.2000 -175.2000   
11       Merauke      ID     84.90       broken clouds  -8.4667  140.3333   
15        Atuona      PF     77.88           clear sky  -9.8000 -139.0333   
17       Rikitea      PF     73.94           clear sky -23.1203 -134.9692   
19      Qui Nhon      VN     82.85     overcast clouds  13.7667  109.2333   
23  Ponta Do Sol      PT     74.71       broken clouds  32.6667  -17.1000   
26      Acapulco      MX     82.22       broken clouds  16.8634  -99.8901   
30     Caravelas      BR     74.46           clear sky -17.7125  -39.2481   
38     San Borja      BO     75.96          few clouds -14.8167  -66.8500   

   Hotel Name  
5              
9              
11             
15             
17             
19             
23             
26             
30             
38

In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

City Country  Max Temp Current Description      Lat       Lng  \
5      Airai      TL     76.37       broken clouds  -8.9266  125.4092   
9      Vaini      TO     82.56    scattered clouds -21.2000 -175.2000   
15    Atuona      PF     77.88           clear sky  -9.8000 -139.0333   
17   Rikitea      PF     73.94           clear sky -23.1203 -134.9692   
19  Qui Nhon      VN     82.85     overcast clouds  13.7667  109.2333   

                Hotel Name  
5           Juvinal Martin  
9      Keleti Beach Resort  
15             Villa Enata  
17         People ThankYou  
19  Hoang Yen Canary Hotel

In [14]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))